# PROYEK ANALISIS DATA: BIKE SHARING DATASET

Nama : Wisnu Purnomo
Email : purnomowisnu08@gmail.com

MENENTUKAN PERTANYAAN BISNIS

Pertanyaan 1 : Pada musim apa kebanyakan orang melakukan rental sepeda? dan mengapa pada musim ini banyak orang merental sepeda?

Pertanyaan 2 : Bagaimana hubungan regresi antara suhu udara dengan total rental sepeda?

### IMPORT LIBRARY YANG DIBUTUHKAN

In [21]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.linear_model import LinearRegression
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt

## DATA WRANGLING

### DATA GATHERING

In [6]:
# import dataset

dataset = pd.read_csv('day.csv')
dataset.head(5)

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,6,0,2,0.344167,0.363625,0.805833,0.160446,331,654,985
1,2,2011-01-02,1,0,1,0,0,0,2,0.363478,0.353739,0.696087,0.248539,131,670,801
2,3,2011-01-03,1,0,1,0,1,1,1,0.196364,0.189405,0.437273,0.248309,120,1229,1349
3,4,2011-01-04,1,0,1,0,2,1,1,0.200000,0.212122,0.590435,0.160296,108,1454,1562
4,5,2011-01-05,1,0,1,0,3,1,1,0.226957,0.229270,0.436957,0.186900,82,1518,1600


### ASSESSING DATA

In [8]:
# melihat tipe kolom dataset

dataset.dtypes

instant         int64
dteday         object
season          int64
yr              int64
mnth            int64
holiday         int64
weekday         int64
workingday      int64
weathersit      int64
temp          float64
atemp         float64
hum           float64
windspeed     float64
casual          int64
registered      int64
cnt             int64
dtype: object

In [9]:
# melihat deskripsi statistik dataset

dataset.describe()

,instant,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
count,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000
mean,366.000000,2.496580,0.500684,6.519836,0.028728,2.997264,0.683995,1.395349,0.495385,0.474354,0.627894,0.190486,848.176471,3656.172367,4504.348837
std,211.165812,1.110807,0.500342,3.451913,0.167155,2.004787,0.465233,0.544894,0.183051,0.162961,0.142429,0.077498,686.622488,1560.256377,1937.211452
min,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.059130,0.079070,0.000000,0.022392,2.000000,20.000000,22.000000
25%,183.500000,2.000000,0.000000,4.000000,0.000000,1.000000,0.000000,1.000000,0.337083,0.337842,0.520000,0.134950,315.500000,2497.000000,3152.000000
50%,366.000000,3.000000,1.000000,7.000000,0.000000,3.000000,1.000000,1.000000,0.498333,0.486733,0.626667,0.180975,713.000000,3662.000000,4548.000000
75%,548.500000,3.000000,1.000000,10.000000,0.000000,5.000000,1.000000,2.000000,0.655417,0.608602,0.730209,0.233214,1096.000000,4776.500000,5956.000000
max,731.000000,4.000000,1.000000,12.000000,1.000000,6.000000,1.000000,3.000000,0.861667,0.840896,0.972500,0.507463,3410.000000,6946.000000,8714.000000


In [10]:
# cek missing value

dataset.isnull().sum()

instant       0
dteday        0
season        0
yr            0
mnth          0
holiday       0
weekday       0
workingday    0
weathersit    0
temp          0
atemp         0
hum           0
windspeed     0
casual        0
registered    0
cnt           0
dtype: int64

### DATA CLEANING

In [18]:
# mengambil kolom yang dibutuhkan saja

dataset.drop(['yr', 'atemp', 'mnth', 'weekday', 'workingday', 'weathersit', 'windspeed', 'hum', 'instant'], axis=1, inplace=True)

In [19]:
dataset

,dteday,season,holiday,temp,casual,registered,cnt
0,2011-01-01,1,0,0.344167,331,654,985
1,2011-01-02,1,0,0.363478,131,670,801
2,2011-01-03,1,0,0.196364,120,1229,1349
3,2011-01-04,1,0,0.200000,108,1454,1562
4,2011-01-05,1,0,0.226957,82,1518,1600
...,...,...,...,...,...,...,...
726,2012-12-27,1,0,0.254167,247,1867,2114
727,2012-12-28,1,0,0.253333,644,2451,3095
728,2012-12-29,1,0,0.253333,159,1182,1341
729,2012-12-30,1,0,0.255833,364,1432,1796


## EXPLORATORY DATA ANALYSIS (EDA)

In [20]:
# melihat musim yang paling banyak muncul

dataset['season'].value_counts()

season
3    188
2    184
1    181
4    178
Name: count, dtype: int64

In [27]:
# mendefinisikan variabel x dan y

x = np.array(dataset['temp']).reshape(-1, 1)
y = dataset['cnt']

In [23]:
# membuat regressor

regressor = LinearRegression()

In [28]:
# melakukan model fit

regressor.fit(x, y)

LinearRegression()

In [29]:
# melakukan prediksi model

y_pred = regressor.predict(x)

In [30]:
print(regressor.score(x, y))

0.3937487313729241


## CONCLUSION

Berdasarkan EDA yang sudah dilakukan, dapat diambil dua kesimpulan pada analisis ini yaitu:

1. Musim yang paling banyak orang melakukan rental sepeda adalah musim Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds. Bisa dikatakan musim ini adalah musim dimana banyak salju dan hujan, udaranya dingin sehingga orang akan memilih naik sepeda saja untuk menghangatkan badan karena mengayuh sepeda sama hal nya dengan olahraga yang bisa membuat badan menjadi hangat. 

2. Nilai R2 dari hasil regresi linier antara variabel temperature dengan total rental sepeda adalah 0.39 atau 39%. Hal ini menunjukan bahwa variabel temperature hanya bisa menjelaskan sebesar 39% variabilitas dalam total rental sepeda, dan selebihnya dijelaskan oleh variabel lain yang tidak dimasukan ke dalam model.